In [ ]:
import numpy as np
import os
import tensorflow as tf

###### Do not modify here ###### 

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

# training on MNIST but only on digits 0 to 4
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]

###### Do not modify here ######

# one_hot encoding 0, 1, 2, 3, 4 for all labels
def one_hot_encoding(y):
    tmp_y = np.zeros([y.shape[0], 5])
    for i in range(y.shape[0]):
        tmp_y[i][y[i]] = 1
    return tmp_y

# split dataset into training set and one validation set
def split_folds(indices, Inputs, Labels, cross_validation, fold):
    n = Inputs.shape[0]
    if fold == cross_validation:
        validation_size = n - (int(n/cross_validation) * (cross_validation-1))
        X_train_idx, X_validate_idx = indices[:(n-validation_size)], indices[(n-validation_size):]
        y_train_idx, y_validate_idx = indices[:(n-validation_size)], indices[(n-validation_size):]
    else:
        validation_size = int(n/cross_validation)
        X_train_idx, X_validate_idx = np.concatenate((indices[:validation_size*(fold-1)], indices[validation_size*fold:]), axis=0), indices[(validation_size*(fold-1)):(validation_size*fold)]
        y_train_idx, y_validate_idx = np.concatenate((indices[:validation_size*(fold-1)], indices[validation_size*fold:]), axis=0), indices[(validation_size*(fold-1)):(validation_size*fold)]
    X_train, X_validate = np.array(Inputs[X_train_idx,:]), np.array(Inputs[X_validate_idx,:])
    y_train, y_validate = np.array(Labels[y_train_idx,:]), np.array(Labels[y_validate_idx,:])
    return X_train, y_train, X_validate, y_validate

def train(X_train, y_train, X_validate, y_validate, optimizer, epoch_bound, stop_threshold, batch_size, testing=False):

    global saver
    
    early_stop = 0
    winner_accuracy = 0.0
    winner_precision = 0.0
    winner_recall = 0.0
    
    for epoch in range(epoch_bound):

        # randomize training set
        indices_training = np.random.permutation(X_train.shape[0])
        X_train, y_train = X_train[indices_training,:], y_train[indices_training,:]

        # split training set into multiple mini-batches and start training
        total_batches = int(X_train.shape[0] / batch_size)
        for batch in range(total_batches):
            if batch == total_batches - 1:
                sess.run(optimizer, feed_dict={x: X_train[batch*batch_size:], y: y_train[batch*batch_size:]})
            else:
                sess.run(optimizer, feed_dict={x: X_train[batch*batch_size : (batch+1)*batch_size], y: y_train[batch*batch_size : (batch+1)*batch_size]})

        # compute validation accuracy
        cur_accuracy, cur_precision, cur_recall = evaluate(X_validate, y_validate)

        # If the accuracy rate does not increase for many times, it will early stop epochs-loop 
        if winner_accuracy < cur_accuracy:
            winner_accuracy = cur_accuracy
            winner_precision = cur_precision
            winner_recall = cur_recall
            # save best model in testing phase
            if testing == True:
                save_path = saver.save(sess, "./saved_model/Team35_HW2.ckpt")
        else:
            early_stop += 1
        if early_stop == stop_threshold:
            break

    return winner_accuracy, winner_precision, winner_recall

# evaluate model: compute accuracy, precision, recall
def evaluate(Inputs, Labels):
    global outputs
    y_predict = sess.run(outputs, feed_dict={x: Inputs, y: Labels})
    correct_prediction = tf.equal(tf.argmax(y_predict, 1), tf.argmax(Labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")
    acc = sess.run(accuracy, feed_dict={x: Inputs, y:Labels})
    
    all_precision = []
    all_recall = []
    for i in range(Labels.shape[1]):
        pred_y = tf.argmax(y_predict, 1)
        true_y = tf.argmax(Labels, 1)

        pred_y_equal_i = tf.cast(tf.equal(pred_y, tf.constant(np.full((pred_y.shape), i), dtype=tf.int64)), tf.float32)
        true_y_equal_i = tf.cast(tf.equal(true_y, tf.constant(np.full((true_y.shape), i), dtype=tf.int64)), tf.float32)

        TP_TF = tf.count_nonzero(pred_y_equal_i)
        TP_TN = tf.count_nonzero(true_y_equal_i)
        TP = tf.count_nonzero(tf.multiply(pred_y_equal_i,true_y_equal_i))
        
        precision = tf.divide(TP,TP_TF, name="precision")
        recall = tf.divide(TP, TP_TN, name="recall")
        
        all_precision.append(sess.run(precision))
        all_recall.append(sess.run(recall))
        
    return acc, all_precision, all_recall

########## Data Processing ##########
# one_hot encoding 0, 1, 2, 3, 4 for all labels
y_train1 = one_hot_encoding(y_train1)
y_valid1 = one_hot_encoding(y_valid1)
y_test1 = one_hot_encoding(y_test1)

Inputs = np.concatenate((X_train1, X_valid1), axis=0)
Labels = np.concatenate((y_train1, y_valid1), axis=0)
########## Data Processing ##########

########## Define hyper-parameters ##########
learning_rate = 0.001
dropout_rate = 0.1
batch_size = 512
epoch_bound = 500
cross_validation = 5
stop_threshold = 20
########## Define hyper-parameters ##########

########## Build DNN ##########
# define placeholder for inputs and labels
x = tf.placeholder(tf.float32, [None, Inputs.shape[1]], name="X")
y = tf.placeholder(tf.float32, [None, Labels.shape[1]], name="y")

# add 5 hidden layers with 128 neurals each
h1 = tf.layers.dense(inputs=x,  units=128, activation=tf.nn.elu, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(), bias_initializer=tf.contrib.layers.xavier_initializer(), name="dnn_h1")
h2 = tf.layers.dense(inputs=h1, units=128, activation=tf.nn.elu, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(), bias_initializer=tf.contrib.layers.xavier_initializer(), name="dnn_h2")
h3 = tf.layers.dense(inputs=h2, units=128, activation=tf.nn.elu, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(), bias_initializer=tf.contrib.layers.xavier_initializer(), name="dnn_h3")
h4 = tf.layers.dense(inputs=h3, units=128, activation=tf.nn.elu, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(), bias_initializer=tf.contrib.layers.xavier_initializer(), name="dnn_h4")
h5 = tf.layers.dense(inputs=h4, units=128, activation=tf.nn.elu, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(), bias_initializer=tf.contrib.layers.xavier_initializer(), name="dnn_h5")

# add outputs softmax layer with 5 neurals
logits = tf.layers.dense(h5, 5, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(), name="logits")
outputs = tf.nn.softmax(logits, name="Y_probability")

# cross entropy loss function
cross_entropy_loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y), name="loss")

# training iteration and define Adam optimizer with learning rate
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy_loss, name="training_op")
########## Build DNN ##########

########## Training ##########
print("########## Start training with dopout ##########")
# start TF session
sess = tf.Session()
init = tf.global_variables_initializer()

# randomize dataset
indices = np.random.permutation(Inputs.shape[0])

# start cross validation
avg_accuracy = 0.0

for fold in range(1, cross_validation+1):
    
    # init weights
    sess.run(init)
    
    # split inputs into training set and validation set for each fold
    X_train, y_train, X_validate, y_validate = split_folds(indices, Inputs, Labels, cross_validation, fold)
    
    winner_accuracy, winner_precision, winner_recall = train(X_train, y_train, X_validate, y_validate, optimizer, epoch_bound, stop_threshold, batch_size, testing=False)
    
    avg_accuracy += winner_accuracy
    print("########## Fold:", fold, "##########")
    print("Accuracy:", winner_accuracy)
    print("Labels:[0, 1, 2, 3, 4]")
    print("Precision:", winner_precision)
    print("Recall:", winner_recall)

avg_accuracy /= cross_validation
print("Average accuracy of cross validation:", avg_accuracy)
sess.close()
########## Training ##########

########## Testing ##########
print("########## Testing results ##########")
# start TF session
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

# init saver to resrore the best model
saver = tf.train.Saver()

# randomize dataset
indices = np.random.permutation(Inputs.shape[0])
Inputs, Labels = np.array(Inputs[indices,:]), np.array(Labels[indices,:])

# get validation set with the size of a batch
X_train, y_train = Inputs[batch_size:], Labels[batch_size:]
X_validate, y_validate = Inputs[:batch_size], Labels[:batch_size]

# start training all the inputs
winner_accuracy, winner_precision, winner_recall = train(X_train, y_train, X_validate, y_validate, optimizer, epoch_bound, stop_threshold, batch_size, testing=True)

saver.restore(sess, "./saved_model/Team35_HW2.ckpt")
test_accuracy, test_precision, test_recall = evaluate(X_test1, y_test1)

print("Accuracy:", test_accuracy)
print("Labels:[0, 1, 2, 3, 4]")
print("Precision:", test_precision)
print("Recall:", test_recall)
sess.close()
########## Test ##########

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
########## Start training with dopout ##########
########## Fold: 1 ##########
Accuracy: 0.989541
Labels:[0, 1, 2, 3, 4]
Precision: [0.99422918384171477, 0.99485672299779571, 0.97934595524956969, 0.99336650082918743, 0.98470688190314359]
Recall: [0.9983443708609272, 0.98760029175784103, 0.98442906574394462, 0.98519736842105265, 0.9922945205479452]
########## Fold: 2 ##########
Accuracy: 0.991338
Labels:[0, 1, 2, 3, 4]
Precision: [0.99221453287197237, 0.99352983465132994, 0.98329156223893066, 0.99241146711635753, 0.99495374264087466]
Recall: [0.99652476107732402, 0.99711399711399706, 0.98907563025210088, 0.98411371237458189, 0.98913043478260865]
